# Questions 3(c) classfication on MNIST data using SVM

## Basic Setup

In [ ]:
import numpy as np
import matplotlib as plt

!pip install qpsolvers
from qpsolvers import solve_qp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 1.8 MB/s eta 0:00:00


In [ ]:
# Installing libsvm
!pip install libsvm-official
from libsvm.svmutil import *

  Preparing metadata (setup.py) ... done
  Created wheel for libsvm-official: filename=libsvm_official-3.32.0-cp310-cp310-linux_x86_64.whl size=123884 sha256=9d3bf0f7cf41b17a747ba6ad6a1e3b57e16f5de72a1c8033078e0d324a771c75
  Stored in directory: /root/.cache/pip/wheels/61/3b/1b/73bb4869517f96a26c82b47ccdb9ec48f12f4466de2371eff6
Successfully built libsvm-official


/usr/local/lib/python3.10/dist-packages/libsvm/svm.py:147: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def csr_to_problem_jit(l, x_val, x_ind, x_rowptr, prob_val, prob_ind, prob_rowptr, indx_start):


In [ ]:
import requests

try:
    from tqdm import tqdm
except ImportError:
    tqdm = lambda x, total, unit: x
    print('**** Could not import tqdm. Please install tqdm for download progressbars! (pip install tqdm) ****')


try:
    input = raw_input
except NameError:
    pass

download_dict = {
    '1) Kuzushiji-MNIST (10 classes, 28x28, 70k examples)': {
        '1) MNIST data format (ubyte.gz)':
            ['http://codh.rois.ac.jp/kmnist/dataset/kmnist/train-images-idx3-ubyte.gz',
            'http://codh.rois.ac.jp/kmnist/dataset/kmnist/train-labels-idx1-ubyte.gz',
            'http://codh.rois.ac.jp/kmnist/dataset/kmnist/t10k-images-idx3-ubyte.gz',
            'http://codh.rois.ac.jp/kmnist/dataset/kmnist/t10k-labels-idx1-ubyte.gz'],
        '2) NumPy data format (.npz)':
            ['http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-imgs.npz',
            'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-train-labels.npz',
            'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-imgs.npz',
            'http://codh.rois.ac.jp/kmnist/dataset/kmnist/kmnist-test-labels.npz'],
    },
    '2) Kuzushiji-49 (49 classes, 28x28, 270k examples)': {
        '1) NumPy data format (.npz)':
            ['http://codh.rois.ac.jp/kmnist/dataset/k49/k49-train-imgs.npz',
            'http://codh.rois.ac.jp/kmnist/dataset/k49/k49-train-labels.npz',
            'http://codh.rois.ac.jp/kmnist/dataset/k49/k49-test-imgs.npz',
            'http://codh.rois.ac.jp/kmnist/dataset/k49/k49-test-labels.npz'],
    },
    '3) Kuzushiji-Kanji (3832 classes, 64x64, 140k examples)': {
        '1) Folders of images (.tar)':
            ['http://codh.rois.ac.jp/kmnist/dataset/kkanji/kkanji.tar'],
    }

}


def download_list(url_list):
    for url in url_list:
        path = url.split('/')[-1]
        r = requests.get(url, stream=True)
        with open(path, 'wb') as f:
            total_length = int(r.headers.get('content-length'))
            print('Downloading {} - {:.1f} MB'.format(path, (total_length / 1024000)))

            for chunk in tqdm(r.iter_content(chunk_size=1024), total=int(total_length / 1024) + 1, unit="KB"):
                if chunk:
                    f.write(chunk)
    print('All dataset files downloaded!')


def traverse_dict(d):
    print('Please select a download option:')
    keys = sorted(d.keys())
    for key in keys:
        print(key)

    userinput = input('> ').strip()

    try:
        selection = int(userinput) - 1
    except ValueError:
        print('Your selection was not valid')
        traverse_dict(d)
        return

    selected = keys[selection]

    next_level = d[selected]
    if isinstance(next_level, list):
        download_list(next_level)
    else:
        traverse_dict(next_level)

traverse_dict(download_dict)

Please select a download option:
1) Kuzushiji-MNIST (10 classes, 28x28, 70k examples)
2) Kuzushiji-49 (49 classes, 28x28, 270k examples)
3) Kuzushiji-Kanji (3832 classes, 64x64, 140k examples)
> 1
Please select a download option:
1) MNIST data format (ubyte.gz)
2) NumPy data format (.npz)
> 2


100%|██████████| 17954/17954 [00:24<00:00, 733.19KB/s] 


100%|██████████| 30/30 [00:00<00:00, 191.70KB/s]


100%|██████████| 3008/3008 [00:04<00:00, 751.20KB/s]


100%|██████████| 6/6 [00:00<00:00, 11837.17KB/s]

All dataset files downloaded!


In [ ]:
train_images = np.load('kmnist-train-imgs.npz')['arr_0'][:1000]/255
val_images= np.load('kmnist-train-imgs.npz')['arr_0'][1000:]/255
train_labels = np.load('kmnist-train-labels.npz')['arr_0'][:1000]
val_labels=np.load('kmnist-train-labels.npz')['arr_0'][1000:]
test_images = np.load('kmnist-test-imgs.npz')['arr_0']/255
test_labels = np.load('kmnist-test-labels.npz')['arr_0']
train_images.shape, train_labels.shape, val_images.shape,val_labels.shape, test_images.shape, test_labels.shape

((1000, 28, 28), (1000,), (59000, 28, 28), (59000,), (10000, 28, 28), (10000,))

In [ ]:
train_images=train_images.reshape(1000,784)
val_images= val_images.reshape(59000,784)
test_images=test_images.reshape(10000,784)
train_images.shape, test_images.shape, train_labels.shape,  test_labels.shape, val_images.shape,val_labels.shape,

((1000, 784), (10000, 784), (1000,), (10000,), (59000, 784), (59000,))

In [ ]:
def pca(data):
    mean = np.mean(data, axis=0)
    centered_data = data - mean
    covariance = np.cov(centered_data.T)
    eigenvalues, eigenvectors = np.linalg.eig(covariance)
    sort_index = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sort_index]
    eigenvectors = eigenvectors[:, sort_index]
    k = 10
    principal_components = eigenvectors[:, :k]

    transformed_data = centered_data.dot(principal_components)
    return  transformed_data
train_images=pca(train_images)
val_images= pca(val_images)
test_images=pca(test_images)
train_images.shape, test_images.shape, train_labels.shape,  test_labels.shape, val_images.shape,val_labels.shape,

((1000, 10), (10000, 10), (1000,), (10000,), (59000, 10), (59000,))

In [ ]:
class MULTI_SVM():
  def __init__(self, X_train, X_test, y_train, y_test, input, C, degree=None, gamma=None):
    self.X_train = X_train
    self.y_train = y_train
    self.X_test = X_test
    self.y_test = y_test
    self.degree = degree
    self.gamma = gamma
    self.C = C
    self.input = input
    self.kernel = [self.linear_kernel, self.polynomial_kernel, self.rbf_kernel]
    self.n_classes = 10

  def linear_kernel(self, x1, x2):
    return np.dot(x1, x2.T)

  def polynomial_kernel(self, X1, X2):
    return (np.dot(X1, X2.T) + 1) ** self.degree

  def rbf_kernel(self, x1, x2):
    try:
      distance = np.linalg.norm(x1[:, None] - x2, axis=2) ** 2
    except:
      distance = np.linalg.norm(x1 - x2) ** 2
    return np.exp(-self.gamma * distance)

  def prediction(self):
    n_samples, n_features = self.X_train.shape
    y_pred = np.zeros(len(self.y_test))
    for class_index in range(self.n_classes):
      y_train_binary = np.where(self.y_train == class_index, 1, -1)

      K = self.kernel[self.input](self.X_train, self.X_train)
      P = np.outer(y_train_binary, y_train_binary) * K
      q = -np.ones(n_samples)
      G = np.vstack((-np.eye(n_samples), np.eye(n_samples)))
      h = np.hstack((np.zeros(n_samples), np.ones(n_samples) * self.C))
      A = self.y_train.reshape(1, -1).astype('d')
      b = np.zeros(1)
      mu = np.array(solve_qp(P, q, G, h ,A,b,solver='cvxopt')).flatten()

      support_vector_indices = np.where((mu > 1e-10) & (mu < self.C))
      X_support = self.X_train[support_vector_indices]
      y_support = self.y_train[support_vector_indices]
      mu_support = mu[support_vector_indices]

      w = np.sum(mu_support * y_support * X_support.T, axis=1)
      b = np.mean(y_support - np.dot(X_support, w))

      z = np.dot(self.X_test, w) + b
      class_y_pred = np.sign(np.array(z))
      y_pred[class_y_pred == 1] = class_index

    accuracy=np.mean(np.array(y_pred)== np.array(self.y_test))
    return accuracy * 100 , y_pred

## Hard SVM
---------------

#### Linear Kernel

In [ ]:
print("")
print("Validation Data")
print("")
model = MULTI_SVM(train_images, val_images, train_labels,  val_labels, input=0, C=100)
Accuracy , y_pred_Linear_HM_SVM = model.prediction()
print(f"Accuracy for Hard MultiClass SVM  with Linear_kernel is on validation data {Accuracy}%")
print(f"y_prediction 20 sample are {y_pred_Linear_HM_SVM[:20]}")
print("")
print("Test Data")
print("")
model = MULTI_SVM(train_images, test_images, train_labels,  test_labels, input=0, C=100)
Accuracy , y_pred_Linear_HM_SVM = model.prediction()
print(f"Accuracy for Hard MultiClass SVM  with Linear_kernel is on test data {Accuracy}%")
print(f"y_prediction 20 sample are {y_pred_Linear_HM_SVM[:20]}")


Validation Data

Accuracy for Hard MultiClass SVM  with Linear_kernel is on validation data 10.01551724137931%
y_prediction 20 sample are [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

Test Data

Accuracy for Hard MultiClass SVM  with Linear_kernel is on test data 10.0%
y_prediction 20 sample are [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


#### Polynomial Kernel

In [ ]:
degree_values = np.linspace(1,5,5)
results_shape = (len(degree_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(degree_values):
    accuracy, y_pred_Poly_HM_SVM = MULTI_SVM(train_images, val_images, train_labels,  val_labels, input=1, degree=param1_value, C=100).prediction()
    results[param1_idx] = accuracy

best_param1_idx= np.unravel_index(np.argmax(results), results_shape)
best_param1 = degree_values[best_param1_idx]
print(f"Best parameters: Degree_of_polynomial = {best_param1}")
print(f"Accuracy for Hard MultiClass SVM  with Polynomials _kernel is on validation data {results[best_param1_idx]}%")

print(f"y_prediction 20 sample are on validation data {y_pred_Poly_HM_SVM[:20]}")
print("")
print("TEST DATA")
print("")
result=MULTI_SVM(train_images, test_images, train_labels,  test_labels, input=1, degree=best_param1, C=100)
Accuracy , y_pred_poly_HM_SVM = result.prediction()
print(f"Accuracy for Hard MultiClass SVM  with Polynomial_kernel is on test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_poly_HM_SVM[:20]}")

Best parameters: Degree_of_polynomial = 1.0
Accuracy for Hard MultiClass SVM  with Polynomials _kernel is on validation data 10.022033898305084%
y_prediction 20 sample are on validation data [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

TEST DATA

Accuracy for Hard MultiClass SVM  with Polynomial_kernel is on test_data 10.0%
y_prediction 20 sample are on test data [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


#### Rbf Kernel

In [ ]:
gamma_values = np.logspace(-3,1,5)
results_shape = (len(gamma_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(gamma_values):
    accuracy, y_pred_rbf_HM_SVM = MULTI_SVM(train_images, val_images, train_labels,  val_labels, input=2, gamma=param1_value, C=100).prediction()
    results[param1_idx] = accuracy

best_param1_idx = np.unravel_index(np.argmax(results), results_shape)
best_param1 = gamma_values[best_param1_idx]
print(f"Best parameters: gamma_value of Rbf kernel = {best_param1}")
print(f"Accuracy for Hard MultiClass SVM  with Rbf_kernel is on validation data {results[best_param1_idx]}%")
print(f"y_prediction 20 sample are on validation data {y_pred_rbf_HM_SVM[:20]}")
print("")
print("TEST DATA")
print("")
result=MULTI_SVM(train_images, test_images, train_labels,  test_labels, input=2, gamma=best_param1, C=100)
Accuracy , y_pred_rbf_HM_SVM = result.prediction()
print(f"Accuracy for Hard MultiClass SVM  with Rbf_kernel is on test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_rbf_HM_SVM[:20]}")

Best parameters: gamma_value of Rbf kernel = 0.001
Accuracy for Hard MultiClass SVM  with Rbf_kernel is on validation data 10.022033898305084%
y_prediction 20 sample are on validation data [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

TEST DATA

Accuracy for Hard MultiClass SVM  with Rbf_kernel is on test_data 10.0%
y_prediction 20 sample are on test data [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


## SOFT SVM
---------------------------

#### Linear Kernel

In [ ]:
C_values = np.logspace(-3, 1, 5)
results_shape = (len(C_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(C_values):
    accuracy, y_pred_Linear_HM_SVM = MULTI_SVM(train_images, val_images, train_labels,  val_labels, input=0, C=param1_value).prediction()
    results[param1_idx] = accuracy

best_param1_idx= np.unravel_index(np.argmax(results), results_shape)
best_param1 = C_values[best_param1_idx]
print("")
print("Validation DATA")
print("")
print(f"Best parameters: C_parameter = {best_param1}")
print(f"Accuracy for Soft MultiClass SVM  with Linear_kernel is on validation data {results[best_param1_idx]}%")
print(f"y_prediction 20 sample are on validation data {y_pred_Linear_HM_SVM[:20]}")
print("")
print("TEST DATA")
print("")
result=MULTI_SVM(train_images, test_images, train_labels,  test_labels, input=0, C=best_param1)
Accuracy , y_pred_Linear_SM_SVM = result.prediction()
print(f"Accuracy for Soft MultiClass SVM  with Linear_kernel is on test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_Linear_SM_SVM[:20]}")


Validation DATA

Best parameters: C_parameter = 0.001
Accuracy for Soft MultiClass SVM  with Linear_kernel is on validation data 10.01551724137931%
y_prediction 20 sample are on validation data [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]

TEST DATA

Accuracy for Soft MultiClass SVM  with Linear_kernel is on test_data 10.0%
y_prediction 20 sample are on test data [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


#### Polynomial Kernel

In [ ]:
Degree_values = [1,2,3,4,5]
C_values = np.logspace(-3, 1, 5)
results_shape = (len(Degree_values), len(C_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(Degree_values):
    for param2_idx, param2_value in enumerate(C_values):
      accuracy = MULTI_SVM(train_images, val_images, train_labels,  val_labels, input=1, degree=param1_value ,C=param2_value).prediction()
      results[param1_idx, param2_idx] = accuracy[0]

best_param1_idx, best_param2_idx = np.unravel_index(np.argmax(results), results_shape)
best_param1 = Degree_values[best_param1_idx]
best_param2 = C_values[best_param2_idx]
print("")
print("Validation DATA")
print("")
print(f"Best parameters: Degree_of_polynomial = {best_param1}, C_value = {best_param2}")
print(f"Accuracy for Soft MultiClass SVM  with Ploynomail kernel with GridSearch is on validation data {results[best_param1_idx, best_param2_idx]}%")
print("")
print("TEST DATA")
print("")
result=MULTI_SVM(train_images, test_images, train_labels,  test_labels, input=1, degree=best_param1, C=best_param2)
Accuracy , y_pred_poly_SM_SVM = result.prediction()
print(f"Accuracy for Soft MultiClass SVM  with Polynomial_kernel is on test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_poly_SM_SVM[:20]}")



Validation DATA

Best parameters: Degree_of_polynomial = 3, C_value = 0.001
Accuracy for Soft MultiClass SVM  with Ploynomail kernel with GridSearch is on validation data 10.01896551724138%

TEST DATA

Accuracy for Soft MultiClass SVM  with Polynomial_kernel is on test_data 10.0%
y_prediction 20 sample are on test data [4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.]


#### Rbf Kernel

In [ ]:
gamma_values = np.logspace(-4, 0, 5)
C_values = np.logspace(-3, 1, 5)
results_shape = (len(gamma_values), len(C_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(gamma_values):
    for param2_idx, param2_value in enumerate(C_values):
      accuracy = MULTI_SVM(train_images, val_images, train_labels,  val_labels, input=2, gamma=param1_value ,C=param2_value).prediction()
      results[param1_idx, param2_idx] = accuracy[0]

best_param1_idx, best_param2_idx = np.unravel_index(np.argmax(results), results_shape)
best_param1 = gamma_values[best_param1_idx]
best_param2 = C_values[best_param2_idx]
print("")
print("Validation DATA")
print("")
print(f"Best parameters: Gamma_value in Rbf Kernel = {best_param1}, C_value = {best_param2}")
print(f"Accuracy for Soft MultiClass SVM  with Rbf kernel with GridSearch is on validation data {results[best_param1_idx, best_param2_idx]}%")
print("")
print("TEST DATA")
print("")
result=MULTI_SVM(train_images, test_images, train_labels,  test_labels,input=2, gamma=best_param1, C=best_param2)
Accuracy , y_pred_rbf_SM_SVM = result.prediction()
print(f"Accuracy for Soft MultiClass SVM  with Rbf_kernel is on test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_rbf_SM_SVM[:20]}")


Validation DATA

Best parameters: Gamma_value in Rbf Kernel = 0.0001, C_value = 0.001
Accuracy for Soft MultiClass SVM  with Rbf kernel with GridSearch is on validation data 10.022033898305084%

TEST DATA

Accuracy for Soft MultiClass SVM  with Rbf_kernel is on test_data 10.0%
y_prediction 20 sample are on test data [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


## Hard SVM using LibSVm

In [ ]:
class HARD_SVM_Multiclass():
  def __init__(self,X_multi_train,X_multi_test,y_multi_train,y_multi_test):
    self.X_multi_train=X_multi_train
    self.X_multi_test=X_multi_test
    self.y_multi_train=y_multi_train
    self.y_multi_test=y_multi_test
    return

  def Linear_kernal_multiClass(self):
    prob_estimates = []
    models = []
    for class_idx in range(10):
        binary_labels = np.where(self.y_multi_train == class_idx, 1, -1)
        model = svm_train(binary_labels, self.X_multi_train, f"-t 0 -b 1 -c 100")
        models.append(model)
    for new_data_point in self.X_multi_test:
        prob_per_class = []
        for model in models:
            _, _, prob= svm_predict([0], [new_data_point.tolist()], model,"-b 1")
            prob_per_class.append(prob[0][0])
        prob_estimates.append(prob_per_class)
    predicted_class = np.argmax(prob_estimates, axis=1)
    accuracy=np.mean(np.array(predicted_class)== np.array(self.y_multi_test))
    return accuracy*100, predicted_class

  def Poly_kernal_multiClass(self, degree):
    prob_estimates = []
    models = []
    for class_idx in range(10):
        binary_labels = np.where(self.y_multi_train == class_idx, 1, -1)
        model = svm_train(binary_labels, self.X_multi_train, f"-t 1 -d {int(degree)} -b 1 -c 100")
        models.append(model)
    for new_data_point in self.X_multi_test:
        prob_per_class = []
        for model in models:
            _, _, prob= svm_predict([0], [new_data_point.tolist()], model,"-b 1")
            prob_per_class.append(prob[0][0])
        prob_estimates.append(prob_per_class)
    predicted_class = np.argmax(prob_estimates, axis=1)
    accuracy=np.mean(np.array(predicted_class)== np.array(self.y_multi_test))
    return accuracy*100, predicted_class

  def Rbf_kernal_multiClass(self, gamma):
    prob_estimates = []
    models = []
    for class_idx in range(10):
        binary_labels = np.where(self.y_multi_train == class_idx, 1, -1)
        model = svm_train(binary_labels, self.X_multi_train, f"-t 2 -g {gamma} -b 1 -c 100")
        models.append(model)
    for new_data_point in self.X_multi_test:
        prob_per_class = []
        for model in models:
            _, _, prob= svm_predict([0], [new_data_point.tolist()], model,"-b 1")
            prob_per_class.append(prob[0][0])
        prob_estimates.append(prob_per_class)
    predicted_class = np.argmax(prob_estimates, axis=1)
    accuracy=np.mean(np.array(predicted_class)== np.array(self.y_multi_test))
    return accuracy*100, predicted_class

  def Sigmoid_kernal_multiClass(self):
    prob_estimates = []
    models = []
    for class_idx in range(10):
        binary_labels = np.where(self.y_multi_train == class_idx, 1, -1)
        model = svm_train(binary_labels, self.X_multi_train, f"-t 3 -b 1 -c 100")
        models.append(model)
    for new_data_point in self.X_multi_test:
        prob_per_class = []
        for model in models:
            _, _, prob= svm_predict([0], [new_data_point.tolist()], model,"-b 1")
            prob_per_class.append(prob[0][0])
        prob_estimates.append(prob_per_class)

    predicted_class = np.argmax(prob_estimates, axis=1)
    accuracy=np.mean(np.array(predicted_class)== np.array(self.y_multi_test))
    return accuracy*100, predicted_class

#### Linear Kernel

In [ ]:
model = HARD_SVM_Multiclass(train_images, val_images, train_labels, val_labels)
accuracy , Y_pred_Linear_HM_SVM = model.Linear_kernal_multiClass()

model = HARD_SVM_Multiclass(train_images, test_images, train_labels, test_labels)
Accuracy , y_pred_Linear_HM_SVM = model.Linear_kernal_multiClass()


Streaming output truncated to the last 5000 lines.
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (cla

In [ ]:
print("")
print("Validation Data")
print("")
print(f"Accuracy for Hard MultiClass SVM  with Linear_kernel is on validation data {accuracy}%")
print(f"y_prediction 20 sample are {Y_pred_Linear_HM_SVM[:20]}")
print("")
print("Test Data")
print("")
print(f"Accuracy for Hard MultiClass SVM  with Linear_kernel is on test data {Accuracy}%")
print(f"y_prediction 20 sample are {y_pred_Linear_HM_SVM[:20]}")


Validation Data

Accuracy for Hard MultiClass SVM  with Linear_kernel is on validation data 5.428813559322034%
y_prediction 20 sample are [7 8 4 7 6 5 3 6 4 5 3 5 3 7 3 0 5 8 5 4]

Test Data

Accuracy for Hard MultiClass SVM  with Linear_kernel is on test data 9.36%
y_prediction 20 sample are [6 9 7 2 9 6 4 5 5 9 5 9 4 4 7 6 9 5 5 9]


#### Polynomail Kernel

In [ ]:
degree_values = np.linspace(1,5,5)
results_shape = (len(degree_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(degree_values):
    accuracy, Y_pred_Poly_HM_SVM = HARD_SVM_Multiclass(train_images, val_images, train_labels, val_labels).Poly_kernal_multiClass(degree=param1_value)
    results[param1_idx] = accuracy

best_param1_idx= np.unravel_index(np.argmax(results), results_shape)
best_param1 = degree_values[best_param1_idx]

result=HARD_SVM_Multiclass(train_images, test_images, train_labels, test_labels)
Accuracy , y_pred_poly_HM_SVM = result.Poly_kernal_multiClass(degree=best_param1)


Streaming output truncated to the last 5000 lines.
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (cla

In [ ]:
print("")
print("Validation Data")
print("")
print(f"Best parameters: Degree_of_polynomial = {best_param1}")
print(f"Accuracy for Hard MultiClass SVM with Polynomials _kernel is on validation data {results[best_param1_idx]}%")
print(f"y_prediction 20 sample are on validation data {Y_pred_Poly_HM_SVM[:20]}")
print("")
print("TEST DATA")
print("")
print(f"Accuracy for Hard MultiClass SVM with Polynomial_kernel is on test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_poly_HM_SVM[:20]}")


Validation Data

Best parameters: Degree_of_polynomial = 2.0
Accuracy for Hard MultiClass SVM with Polynomials _kernel is on validation data 15.472881355932202%
y_prediction 20 sample are on validation data [2 7 2 7 6 7 2 2 2 3 2 3 3 8 2 0 4 2 2 0]

TEST DATA

Accuracy for Hard MultiClass SVM with Polynomial_kernel is on test_data 13.459999999999999%
y_prediction 20 sample are on test data [8 2 0 4 3 6 4 2 2 1 1 3 3 5 5 2 4 0 5 3]


#### RBF Kernel

In [ ]:
gamma_values = np.logspace(-3, 1, 5)
results_shape = (len(gamma_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(gamma_values):
    accuracy, Y_pred_rbf_HM_SVM = HARD_SVM_Multiclass(train_images, val_images, train_labels, val_labels).Rbf_kernal_multiClass(gamma=param1_value)
    results[param1_idx] = accuracy

best_param1_idx = np.unravel_index(np.argmax(results), results_shape)
best_param1 = gamma_values[best_param1_idx]

result=HARD_SVM_Multiclass(train_images, test_images, train_labels, test_labels)
Accuracy , y_pred_rbf_HM_SVM = result.Rbf_kernal_multiClass(gamma=best_param1)


Streaming output truncated to the last 5000 lines.
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (cla

In [ ]:
print("")
print("Validation Data")
print("")
print(f"Best parameters: gamma_value of Rbf kernel = {best_param1}")
print(f"Accuracy for Hard MultiClass SVM  with Rbf_kernel is on validation data {results[best_param1_idx]}%")
print(f"y_prediction 20 sample are on validation data {Y_pred_rbf_HM_SVM[:20]}")
print("")
print("TEST DATA")
print("")
print(f"Accuracy for Hard MultiClass SVM  with Rbf_kernel is on test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_rbf_HM_SVM[:20]}")


Validation Data

Best parameters: gamma_value of Rbf kernel = 10.0
Accuracy for Hard MultiClass SVM  with Rbf_kernel is on validation data 9.979661016949153%
y_prediction 20 sample are on validation data [9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9]

TEST DATA

Accuracy for Hard MultiClass SVM  with Rbf_kernel is on test_data 10.0%
y_prediction 20 sample are on test data [9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9]


## Soft SVM using LibSvm

In [ ]:
class SOFT_SVM_Multiclass():
  def __init__(self,X_multi_train,X_multi_test,y_multi_train,y_multi_test):
    self.X_multi_train=X_multi_train
    self.X_multi_test=X_multi_test
    self.y_multi_train=y_multi_train
    self.y_multi_test=y_multi_test
    return

  def Linear_kernal_multiClass(self, C):
    prob_estimates = []
    models = []
    for class_idx in range(10):
        binary_labels = np.where(self.y_multi_train == class_idx, 1, -1)
        model = svm_train(binary_labels, self.X_multi_train, f"-c {C} -t 0 -b 1")
        models.append(model)
    for new_data_point in self.X_multi_test:
        prob_per_class = []
        for model in models:
            _, _, prob= svm_predict([0], [new_data_point.tolist()], model,"-b 1")
            prob_per_class.append(prob[0][0])
        prob_estimates.append(prob_per_class)
    predicted_class = np.argmax(prob_estimates, axis=1)
    accuracy=np.mean(np.array(predicted_class)== np.array(self.y_multi_test))
    return accuracy*100, predicted_class

  def Poly_kernal_multiClass(self,degree,C):
    prob_estimates = []
    models = []
    for class_idx in range(10):
        binary_labels = np.where(self.y_multi_train == class_idx, 1, -1)
        model = svm_train(binary_labels, self.X_multi_train, f"-c {C}  -t 1 -d {int(degree)}  -b 1")
        models.append(model)
    for new_data_point in self.X_multi_test:
        prob_per_class = []
        for model in models:
            _, _, prob= svm_predict([0], [new_data_point.tolist()], model,"-b 1")
            prob_per_class.append(prob[0][0])
        prob_estimates.append(prob_per_class)
    predicted_class = np.argmax(prob_estimates, axis=1)
    accuracy=np.mean(np.array(predicted_class)== np.array(self.y_multi_test))
    return accuracy*100, predicted_class

  def Rbf_kernal_multiClass(self,gamma,C):
    prob_estimates = []
    models = []
    for class_idx in range(10):
        binary_labels = np.where(self.y_multi_train == class_idx, 1, -1)
        model = svm_train(binary_labels, self.X_multi_train, f"-c {C}  -t 2 -g {gamma}  -b 1")
        models.append(model)
    for new_data_point in self.X_multi_test:
        prob_per_class = []
        for model in models:
            _, _, prob= svm_predict([0], [new_data_point.tolist()], model,"-b 1")
            prob_per_class.append(prob[0][0])
        prob_estimates.append(prob_per_class)
    predicted_class = np.argmax(prob_estimates, axis=1)
    accuracy=np.mean(np.array(predicted_class)== np.array(self.y_multi_test))
    return accuracy*100, predicted_class

  def Sigmoid_kernal_multiClass(self, C):
    prob_estimates = []
    models = []
    for class_idx in range(10):
        binary_labels = np.where(self.y_multi_train == class_idx, 1, -1)
        model = svm_train(binary_labels, self.X_multi_train, f"-c {C} -t 3 -b 1")
        models.append(model)
    for new_data_point in self.X_multi_test:
        prob_per_class = []
        for model in models:
            _, _, prob= svm_predict([0], [new_data_point.tolist()], model,"-b 1")
            prob_per_class.append(prob[0][0])
        prob_estimates.append(prob_per_class)
    predicted_class = np.argmax(prob_estimates, axis=1)
    accuracy=np.mean(np.array(predicted_class)== np.array(self.y_multi_test))
    return accuracy*100, predicted_class

#### Linear Kernel

In [ ]:
C_values = np.logspace(-2, 1, 5)
results_shape = (len(C_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(C_values):
    accuracy, Y_pred_Linear_HM_SVM = SOFT_SVM_Multiclass(train_images, val_images, train_labels, val_labels).Linear_kernal_multiClass(C=param1_value)
    results[param1_idx] = accuracy

best_param1_idx= np.unravel_index(np.argmax(results), results_shape)
best_param1 = C_values[best_param1_idx]

result=SOFT_SVM_Multiclass(train_images, test_images, train_labels, test_labels)
Accuracy , y_pred_Linear_SM_SVM = result.Linear_kernal_multiClass(C=best_param1)

Streaming output truncated to the last 5000 lines.
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (cla

In [ ]:
print("")
print("Validation DATA")
print("")
print(f"Best parameters: C_parameter = {best_param1}")
print(f"Accuracy for Soft MultiClass SVM  with Linear_kernel is on validation data {results[best_param1_idx]}%")
print(f"y_prediction 20 sample are on validation data {Y_pred_Linear_HM_SVM[:20]}")
print("")
print("TEST DATA")
print("")
print(f"Accuracy for Soft MultiClass SVM  with Linear_kernel is on test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_Linear_SM_SVM[:20]}")


Validation DATA

Best parameters: C_parameter = 0.05623413251903491
Accuracy for Soft MultiClass SVM  with Linear_kernel is on validation data 6.330508474576272%
y_prediction 20 sample are on validation data [7 8 4 7 6 5 3 6 4 3 3 5 3 7 3 0 5 8 5 4]

TEST DATA

Accuracy for Soft MultiClass SVM  with Linear_kernel is on test_data 9.379999999999999%
y_prediction 20 sample are on test data [6 9 7 2 9 6 4 2 5 9 5 9 4 4 7 2 9 3 5 9]


#### Polynomail Kernel

In [ ]:
Degree_values = [1,2,3]
C_values = np.logspace(-2, 1, 5)
results_shape = (len(Degree_values), len(C_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(Degree_values):
    for param2_idx, param2_value in enumerate(C_values):
      accuracy, Y_pred_poly_HM_SVM = SOFT_SVM_Multiclass(train_images, val_images, train_labels, val_labels).Poly_kernal_multiClass(degree=param1_value ,C=param2_value)
      results[param1_idx, param2_idx] = accuracy

best_param1_idx, best_param2_idx = np.unravel_index(np.argmax(results), results_shape)
best_param1 = Degree_values[best_param1_idx]
best_param2 = C_values[best_param2_idx]

result=SOFT_SVM_Multiclass(train_images, test_images, train_labels, test_labels)
Accuracy , y_pred_poly_SM_SVM = result.Poly_kernal_multiClass(degree=best_param1, C=best_param2)

Streaming output truncated to the last 5000 lines.
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (cla

In [ ]:
print("")
print("Validation DATA")
print("")
print(f"Best parameters: Degree_of_polynomial = {best_param1}, C_value = {best_param2}")
print(f"Accuracy for Soft MultiClass SVM  with Ploynomail kernel with GridSearch is on validation data {results[best_param1_idx, best_param2_idx]}%")
print(f"y_prediction 20 sample are on validation data {Y_pred_poly_HM_SVM[:20]}")
print("")
print("TEST DATA")
print("")
print(f"Accuracy for Soft MultiClass SVM  with Polynomial_kernel is on test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_poly_SM_SVM[:20]}")


Validation DATA

Best parameters: Degree_of_polynomial = 2, C_value = 10.0
Accuracy for Soft MultiClass SVM  with Ploynomail kernel with GridSearch is on validation data 16.25593220338983%
y_prediction 20 sample are on validation data [7 7 1 7 6 7 1 7 8 3 7 3 3 8 7 0 4 2 6 1]

TEST DATA

Accuracy for Soft MultiClass SVM  with Polynomial_kernel is on test_data 14.12%
y_prediction 20 sample are on test data [2 2 8 4 3 6 4 2 2 1 1 3 3 5 5 2 4 6 5 3]


#### Rbf Kernel

In [ ]:
gamma_values = np.logspace(-3, 1, 5)
C_values = np.logspace(-2, 1, 5)
results_shape = (len(gamma_values), len(C_values))
results = np.zeros(results_shape, dtype=np.float64)

for param1_idx, param1_value in enumerate(gamma_values):
    for param2_idx, param2_value in enumerate(C_values):
      accuracy, Y_pred_rbf_SM_SVM  = SOFT_SVM_Multiclass(train_images, val_images, train_labels, val_labels).Rbf_kernal_multiClass(gamma=param1_value ,C=param2_value)
      results[param1_idx, param2_idx] = accuracy

best_param1_idx, best_param2_idx = np.unravel_index(np.argmax(results), results_shape)
best_param1 = gamma_values[best_param1_idx]
best_param2 = C_values[best_param2_idx]

result=SOFT_SVM_Multiclass(train_images, test_images, train_labels, test_labels)
Accuracy , y_pred_rbf_SM_SVM = result.Rbf_kernal_multiClass(gamma=best_param1, C=best_param2)


Streaming output truncated to the last 5000 lines.
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (classification)
Accuracy = 0% (0/1) (cla

In [ ]:
print("")
print("Validation DATA")
print("")
print(f"Best parameters: Gamma_value in Rbf Kernel = {best_param1}, C_value = {best_param2}")
print(f"Accuracy for Soft MultiClass SVM  with Rbf kernel with GridSearch is on validation data {results[best_param1_idx, best_param2_idx]}%")
print(f"y_prediction 20 sample are on validation data {Y_pred_rbf_SM_SVM[:20]}")
print("")
print("TEST DATA")
print("")
print(f"Accuracy for Soft MultiClass SVM  with Rbf_kernel is on test_data {Accuracy}%")
print(f"y_prediction 20 sample are on test data {y_pred_rbf_SM_SVM[:20]}")


Validation DATA

Best parameters: Gamma_value in Rbf Kernel = 10.0, C_value = 0.01
Accuracy for Soft MultiClass SVM  with Rbf kernel with GridSearch is on validation data 9.981355932203389%
y_prediction 20 sample are on validation data [9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9]

TEST DATA

Accuracy for Soft MultiClass SVM  with Rbf_kernel is on test_data 10.0%
y_prediction 20 sample are on test data [9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9]
